In [1]:
import os
import ssl
import requests
from io import BytesIO
from urllib import request

from tqdm import tqdm
from PIL import Image

import pandas as pd
import json
import numpy as np
import seaborn as sn
import pickle

from matplotlib import pyplot as plt
from io import StringIO
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
import tensorflow_datasets as tfds

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
import cv2

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [5]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_ds = image_dataset_from_directory(
    "../data_room_classifier/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

test_ds = image_dataset_from_directory(
    "../data_room_classifier/test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)

class_names = train_ds.class_names
NUM_CLASSES = len(class_names)
print(class_names)

test_class_names = test_ds.class_names
print(test_class_names)

Found 3861 files belonging to 18 classes.


2026-01-15 13:42:42.570394: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2026-01-15 13:42:42.570522: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2026-01-15 13:42:42.570527: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2026-01-15 13:42:42.570954: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-01-15 13:42:42.570984: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 621 files belonging to 18 classes.
['closet', 'computerroom', 'corridor', 'dining_room', 'elevator', 'gameroom', 'garage', 'gym', 'kitchen', 'livingroom', 'lobby', 'meeting_room', 'office', 'pantry', 'restaurant', 'restaurant_kitchen', 'tv_studio', 'waitingroom']
['closet', 'computerroom', 'corridor', 'dining_room', 'elevator', 'gameroom', 'garage', 'gym', 'kitchen', 'livingroom', 'lobby', 'meeting_room', 'office', 'pantry', 'restaurant', 'restaurant_kitchen', 'tv_studio', 'waitingroom']


In [6]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
])

In [7]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models

base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

# Freeze base model
base_model.trainable = False

model = models.Sequential([
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')  # Now this matches dataset
])



model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,049,571 (15.45 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,049,571 (15.45 MB)

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',  # Changed from categorical
    metrics=['accuracy']
)


In [9]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,049,571 (15.45 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,049,571 (15.45 MB)

In [10]:
EPOCHS = 15

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=EPOCHS
)


Epoch 1/15


2026-01-15 13:42:48.572636: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


121/121 ━━━━━━━━━━━━━━━━━━━━ 84s 606ms/step - accuracy: 0.5913 - loss: 1.3589 - val_accuracy: 0.7665 - val_loss: 0.7454
Epoch 2/15
121/121 ━━━━━━━━━━━━━━━━━━━━ 65s 529ms/step - accuracy: 0.7415 - loss: 0.8376 - val_accuracy: 0.7939 - val_loss: 0.6803
Epoch 3/15
121/121 ━━━━━━━━━━━━━━━━━━━━ 67s 554ms/step - accuracy: 0.7690 - loss: 0.7455 - val_accuracy: 0.7826 - val_loss: 0.6999
Epoch 4/15
121/121 ━━━━━━━━━━━━━━━━━━━━ 61s 502ms/step - accuracy: 0.7884 - loss: 0.6700 - val_accuracy: 0.7874 - val_loss: 0.7190
Epoch 5/15
121/121 ━━━━━━━━━━━━━━━━━━━━ 167s 1s/step - accuracy: 0.7881 - loss: 0.6754 - val_accuracy: 0.7939 - val_loss: 0.6740
Epoch 6/15
121/121 ━━━━━━━━━━━━━━━━━━━━ 59s 487ms/step - accuracy: 0.8125 - loss: 0.5882 - val_accuracy: 0.8019 - val_loss: 0.6653
Epoch 7/15
121/121 ━━━━━━━━━━━━━━━━━━━━ 455s 4s/step - accuracy: 0.8127 - loss: 0.6019 - val_accuracy: 0.8035 - val_loss: 0.6245
Epoch 8/15
121/121 ━━━━━━━━━━━━━━━━━━━━ 60s 493ms/step - accuracy: 0.8179 - loss: 0.5478 - val_acc

In [11]:
history

In [12]:
#####Fine tuning the model

In [13]:
# Unfreeze top layers
base_model.trainable = True

for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

fine_tune_epochs = 10

history_fine = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=fine_tune_epochs
)


Epoch 1/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - accuracy: 0.6996 - loss: 0.9651 - val_accuracy: 0.8100 - val_loss: 0.7128
Epoch 2/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 127s 1s/step - accuracy: 0.8190 - loss: 0.5861 - val_accuracy: 0.8084 - val_loss: 0.6712
Epoch 3/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 121s 1s/step - accuracy: 0.8526 - loss: 0.4478 - val_accuracy: 0.8084 - val_loss: 0.6627
Epoch 4/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 124s 1s/step - accuracy: 0.8700 - loss: 0.3846 - val_accuracy: 0.8068 - val_loss: 0.6595
Epoch 5/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 120s 990ms/step - accuracy: 0.8964 - loss: 0.3033 - val_accuracy: 0.8084 - val_loss: 0.6751
Epoch 6/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 123s 1s/step - accuracy: 0.9049 - loss: 0.2778 - val_accuracy: 0.8148 - val_loss: 0.6577
Epoch 7/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.9127 - loss: 0.2621 - val_accuracy: 0.8148 - val_loss: 0.6461
Epoch 8/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 125s 1s/step - accuracy: 0.9231 - loss: 0.2282 - val_a

In [14]:
history_fine

In [15]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(np.argmax(labels.numpy(), axis=1))
    y_pred.extend(np.argmax(preds, axis=1))

print(classification_report(y_true, y_pred, target_names=class_names))

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 777ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 386ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 348ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 333ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 401ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
                    precision    recall  f1-score   support

            closet       0.97      1.00      0.99        38
      computerroom       0.62      1.00      0.77         5
          corridor       0.38      1.00      

2026-01-15 14:38:35.935852: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [16]:
model.save("../_models/cnn_room_classifier_effnet.keras")

In [41]:
model_test.input_shape

(None, 224, 224, 3)

In [18]:
from tensorflow.keras.preprocessing import image

In [19]:
IMG_SIZE = (224, 224)

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    return img_array

In [20]:
class_names = [
    'closet', 'computerroom', 'corridor', 'dining_room', 'elevator', 
     'gameroom', 'garage', 'gym', 'kitchen', 'livingroom', 'lobby', 'meeting_room', 
     'office', 'pantry', 'restaurant', 'restaurant_kitchen', 'tv_studio', 'waitingroom']


In [51]:
img_path = "../data_room_classifier/living_room.jpg"

img_tensor = preprocess_image(img_path)
pred = model_test.predict(img_tensor)

predicted_class = class_names[np.argmax(pred)]
confidence = np.max(pred)

print(f"Predicted Room: {predicted_class}")
print(f"Confidence: {confidence:.2%}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
Predicted Room: livingroom
Confidence: 60.15%


In [50]:
img_path = "../data_room_classifier/dining_room.jpg"

img_tensor = preprocess_image(img_path)
pred = model_test.predict(img_tensor)

predicted_class = class_names[np.argmax(pred)]
confidence = np.max(pred)

print(f"Predicted Room: {predicted_class}")
print(f"Confidence: {confidence:.2%}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
Predicted Room: restaurant
Confidence: 48.94%


In [52]:
img_path = "../data_room_classifier/Kitchen.jpg"

img_tensor = preprocess_image(img_path)
pred = model_test.predict(img_tensor)

predicted_class = class_names[np.argmax(pred)]
confidence = np.max(pred)

print(f"Predicted Room: {predicted_class}")
print(f"Confidence: {confidence:.2%}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step
Predicted Room: kitchen
Confidence: 99.98%


In [53]:
img_path = "../data_room_classifier/Kitchen1.jpg"

img_tensor = preprocess_image(img_path)
pred = model_test.predict(img_tensor)

predicted_class = class_names[np.argmax(pred)]
confidence = np.max(pred)

print(f"Predicted Room: {predicted_class}")
print(f"Confidence: {confidence:.2%}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
Predicted Room: kitchen
Confidence: 99.87%


In [54]:
img_path = "../data_room_classifier/Kitchen2.jpg"

img_tensor = preprocess_image(img_path)
pred = model_test.predict(img_tensor)

predicted_class = class_names[np.argmax(pred)]
confidence = np.max(pred)

print(f"Predicted Room: {predicted_class}")
print(f"Confidence: {confidence:.2%}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Predicted Room: kitchen
Confidence: 99.88%


In [42]:
import tensorflow as tf
import tf2onnx


model_test = tf.keras.models.load_model("../_models/cnn_room_classifier_effnet.keras")

@tf.function
def serving_default(input_tensor):
    return model_test(input_tensor, training=False) 

input_signature = [tf.TensorSpec(model_test.input_shape, tf.float32, name='input')]

output_path = "../_models/cnn_room_classifier_effnet_fixed.onnx"

model_proto, _ = tf2onnx.convert.from_function(
    serving_default,
    input_signature=input_signature,
    opset=13,
    output_path=output_path
)

print(f"Success! Cleaned model saved to {output_path}")

2026-01-15 18:39:16.942230: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2026-01-15 18:39:16.945030: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-01-15 18:39:16.945043: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2026-01-15 18:39:22.207340: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-01-15 18:39:22.207432: I tensorflow/core/com

Success! Cleaned model saved to ../_models/cnn_room_classifier_effnet_fixed.onnx


In [60]:
import onnxruntime as ort
import numpy as np
from tensorflow.keras.preprocessing import image

session = ort.InferenceSession(
    "../_models/cnn_room_classifier_effnet.onnx",
    providers=["CPUExecutionProvider"]
)

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name


In [61]:
def preprocess_image_onnx(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    return img_array.astype(np.float32)

In [65]:
img_path = "../data_room_classifier/dining_room.jpg"

img_tensor = preprocess_image_onnx(img_path)
mean = np.array([0.485, 0.456, 0.406], dtype=np.float32).reshape(1,1,1,3)  
std = np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape(1,1,1,3)

input_feed = {
    'input': img_tensor,  # Or whatever the image input name is
    'sequential_1_1/efficientnetb0_1/normalization_1/Sub/y:0': mean,
    'sequential_1_1/efficientnetb0_1/normalization_1/Sqrt/x:0': std
}
pred = session.run([output_name], input_feed)[0]

predicted_class = class_names[np.argmax(pred)]
confidence = np.max(pred)

print("Predicted Room:", predicted_class)
print("Confidence:", f"{confidence:.2%}")

Predicted Room: tv_studio
Confidence: 22.63%


In [64]:
img_path = "../data_room_classifier/living_room.jpg"

img_tensor = preprocess_image_onnx(img_path)
mean = np.array([0.485, 0.456, 0.406], dtype=np.float32).reshape(1,1,1,3)  
std = np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape(1,1,1,3)

input_feed = {
    'input': img_tensor,  
    'sequential_1_1/efficientnetb0_1/normalization_1/Sub/y:0': mean,
    'sequential_1_1/efficientnetb0_1/normalization_1/Sqrt/x:0': std
}
pred = session.run([output_name], input_feed)[0]

predicted_class = class_names[np.argmax(pred)]
confidence = np.max(pred)

print("Predicted Room:", predicted_class)
print("Confidence:", f"{confidence:.2%}")

Predicted Room: dining_room
Confidence: 39.79%


In [66]:
img_path = "../data_room_classifier/Kitchen.jpg"

img_tensor = preprocess_image_onnx(img_path)
mean = np.array([0.485, 0.456, 0.406], dtype=np.float32).reshape(1,1,1,3)  
std = np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape(1,1,1,3)

input_feed = {
    'input': img_tensor,  
    'sequential_1_1/efficientnetb0_1/normalization_1/Sub/y:0': mean,
    'sequential_1_1/efficientnetb0_1/normalization_1/Sqrt/x:0': std
}
pred = session.run([output_name], input_feed)[0]

predicted_class = class_names[np.argmax(pred)]
confidence = np.max(pred)

print("Predicted Room:", predicted_class)
print("Confidence:", f"{confidence:.2%}")

Predicted Room: kitchen
Confidence: 98.40%


In [67]:
img_path = "../data_room_classifier/Kitchen1.jpg"

img_tensor = preprocess_image_onnx(img_path)
mean = np.array([0.485, 0.456, 0.406], dtype=np.float32).reshape(1,1,1,3)  
std = np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape(1,1,1,3)

input_feed = {
    'input': img_tensor,  
    'sequential_1_1/efficientnetb0_1/normalization_1/Sub/y:0': mean,
    'sequential_1_1/efficientnetb0_1/normalization_1/Sqrt/x:0': std
}
pred = session.run([output_name], input_feed)[0]

predicted_class = class_names[np.argmax(pred)]
confidence = np.max(pred)

print("Predicted Room:", predicted_class)
print("Confidence:", f"{confidence:.2%}")

Predicted Room: kitchen
Confidence: 91.62%


In [68]:
img_path = "../data_room_classifier/Kitchen2.jpg"

img_tensor = preprocess_image_onnx(img_path)
mean = np.array([0.485, 0.456, 0.406], dtype=np.float32).reshape(1,1,1,3)  
std = np.array([0.229, 0.224, 0.225], dtype=np.float32).reshape(1,1,1,3)

input_feed = {
    'input': img_tensor,  
    'sequential_1_1/efficientnetb0_1/normalization_1/Sub/y:0': mean,
    'sequential_1_1/efficientnetb0_1/normalization_1/Sqrt/x:0': std
}
pred = session.run([output_name], input_feed)[0]

predicted_class = class_names[np.argmax(pred)]
confidence = np.max(pred)

print("Predicted Room:", predicted_class)
print("Confidence:", f"{confidence:.2%}")

Predicted Room: kitchen
Confidence: 95.47%
